In [1]:
# TensorFlow and tf.keras
# Need to install tensorflow and keras library if not already installed
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator #For processing the images so they can be run through the network
from tensorflow.keras.models import Sequential #Needed for building the network nodes
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense #Needed for building the Coonvultional layers
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image



2024-03-27 09:50:40.758689: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-27 09:50:40.929981: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-27 09:50:41.502623: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-27 09:50:43.071739: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
from google.colab import auth, drive #For accessing the database through drive
auth.authenticate_user()
drive.mount("/content/drive", force_remount=True)

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# Define paths to the training and testing data directories

datasets = "Path for dataset" ###Need to change for the right path"

train_data_dir = os.path.join(datasets, "training")
test_data_dir = os.path.join(datasets, "validation")

print(os.listdir(train_data_dir))
print(os.listdir(test_data_dir))
#Incase we need access to specific cat breed folders I will make variables with directories to each one
"""See training folders"""
# Need to change with right breeds
training_persion = os.path.join(train_data_dir, "Bulldog")
training_abyssian = os.path.join(train_data_dir, "German Shepherd")
training_munchkin = os.path.join(train_data_dir, "Golden Retriever")
training_toyger = os.path.join(train_data_dir, "Pitbull")

"""See testing folders"""
testing_munchkin = os.path.join(test_data_dir, "Bulldog")
testing_toyger = os.path.join(test_data_dir, "German Shepherd")
testing_abyssian = os.path.join(test_data_dir, "Golden Retriever")
testing_persian = os.path.join(test_data_dir, "Pitbull")

In [ ]:

# Define image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32 # determes the number of samples processed by the model in each training iteration

# Create data generors with augmentation for training and validation data
train_datagen = ImageDataGenerator(
    rescale=1./255,             # Rescale pixel values to [0,1]
    shear_range=0.2,            # Shearing transformation
    zoom_range=0.2,             # Zooming transformation
    horizontal_flip=True)       # Horizontal flipping

test_datagen = ImageDataGenerator(rescale=1./255)  # Rescale pixel values for testing data

In [ ]:
"""
train_generator is an object that will continuously generate batches of training data from the specified directory
with images resized to  dimensions and one encoded class
labels suitable for multi-class classification tasks.

- train_datagen is a instance of the ImageDataGenerator used to prepocess the images
- flow_from_directory method of the ImageDataGenerator class
- class_mode specifies the type of labels returned by the generator
"""
train_generator = train_datagen.flow_from_directory(
    train_data_dir, target_size=(img_width, img_height), batch_size=batch_size,
    class_mode='categorical')  # Use categorical labels for multi-class classification

validation_generator = test_datagen.flow_from_directory(
    test_data_dir, target_size=(img_width, img_height), batch_size=batch_size,
    class_mode='categorical')

In [ ]:
# Define the CNN model
model = Sequential()  # Create a sequential model

# Adding Convolutional layers with MaxPooling
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.summary()

In [ ]:
# Flatten the output from Convolutional layers for input to Dense layers
model.add(Flatten())

# Add Dense layers
model.add(Dense(512, activation='relu'))
model.add(Dense(len(train_generator.class_indices), activation='softmax'))  # Output layer

model.summary()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,  # Number of steps per epoch,
    epochs=15,                                              # Number of epochs
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)  # Number of validation steps

In [ ]:
# Save the trained model/ Donot run unless acuraccy over 80%
model.save('Dog_breed_classifier_model.h5')

In [ ]:
# Evaluate the Accuracy of the model
# Could not run because I forgot the passwowrd
# validation_generator yields batches of images and their corresponding labels from the specified directory.IF we do not like how it works we can change it
test_loss, test_accuracy = model.evaluate(validation_generator, verbose = 2)

# Printing the accuracy
print('\nTest Accuracy:', test_accuracy)

In [ ]:
#Prediction
probability_model = tf.keras.Sequential([model,
                                         tf.keras.layers.Softmax()])
predictions = probability_model.predict(test_data_dir)

# need a way for us to see it

In [ ]:
#Loading the model
model = load_model('cog_breed_classifier_model.h5')

In [ ]:
# Load and preprocess the input image in order for it to be the same way as the training set
img_path = 'We need to specify the path to the image input'
img = image.load_img(img_path, target_size=(150, 150))  # Resize image to match model input dimensions
img_array = image.img_to_array(img)
img_array = img_array / 255.0  # Normalize pixel values
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension



In [ ]:
#Interpreting Predictions
predictions = model.predict(img_array)

In [ ]:
# Interpreting the prediction
predicted_class_index = np.argmax(predictions) # argmax is used to find the index of the class with the highes probability.
predicted_class_probability = np.max(predictions) #use max to show the one with the highest probability
class_labels = ["Bulldog", "German Shepherd","Golden Retriever","Pitbull"]
predicted_class_label = class_labels[predicted_class_index]

In [ ]:
print('Predicted Class: ', predicted_class_label)
print('Predicted Probability: ', predicted_class_probability)